## Cargado de DataFrame

In [0]:
df = spark.read.table("samples.bakehouse.sales_customers")
df_selected = df.select("*")

df_limited = df.limit(4)

df_limited.display()


## DISTINCT

In [0]:
df_distinct = df_selected.select("country").distinct()
df_distinct.display()

## Where / Filter

In [0]:
%sql
select *
from samples.bakehouse.sales_customers
where 1 = 1
and country = 'Japan'
and gender ='female'
limit 5

In [0]:
from pyspark.sql.functions import col
df_filtered = df_selected.filter(
    (col("country") == "USA") &
    (col("gender") == "female")
    )
display(df_filtered.limit(5))

## Order BY + LIMIT

In [0]:
%sql
select *
from samples.bakehouse.sales_customers
order by first_name ASC
limit 5

In [0]:
df_ordered = df.orderBy(col("first_name").asc())
display(df_ordered.limit(5))

## CREAR COLUMNA

In [0]:
%sql
select 
        concat(first_name, ' ',last_name) as full_name,
        concat_ws(', ', address, city, state, country) as address,
        35 as age,
        "random comment" as comment,
        True as is_true
from samples.bakehouse.sales_customers
limit 5


In [0]:
from pyspark.sql.functions import col, lit, concat
df_plus_columns = df.select(
    concat(col("first_name"),lit(" "),col("last_name")).alias("full_name"),
    lit(True).alias("is_true"),
    lit("random comment").alias("comment")
).limit(5)

display(df_plus_columns)

## GROUP BY + AGREGATION

In [0]:
%sql
select country, count(1) as cantidad
from samples.bakehouse.sales_franchises
group by country
order by cantidad DESC

In [0]:
from pyspark.sql.functions import count

df_grouped = (
    df
    .groupBy("country")
    .agg(count("*").alias("count"))
    .orderBy(col("count").desc())
)

display(df_grouped.limit(10))


## JOIN 


In [0]:
%sql
select 
  concat(c.first_name, ' ', c.last_name) as full_name,
  c.country,
  c.gender,
  s.product,
  s.quantity,
  s.paymentMethod,
  s.dateTime
from samples.bakehouse.sales_transactions s
JOIN samples.bakehouse.sales_customers c
ON s.customerID = c.customerID
limit 5

In [0]:
df_sales = spark.read.table("samples.bakehouse.sales_transactions")

df_joined = (
    df
    .join(df_sales, on="customerID", how="inner")
    .select(
        "customerID",
        "first_name",
        "last_name",
        "country",
        "transactionID",
        "quantity",
        "unitPrice",
        "totalPrice",
        "paymentMethod"
    )
)

display(df_joined.limit(5))

## Casting de Datos

In [0]:
%sql
select cast(transactionID as string) as transactionID,
      CAST(customerID as string) as customerID,
      CAST(franchiseID as string) as franchiseID,
      CAST(dateTime as string) as dateTime,
      CAST(product as string) as product 
from samples.bakehouse.sales_transactions
limit 6

In [0]:
from pyspark.sql.functions import col
df_casted = df_sales.select(
    col("transactionID").cast("bigInt"),
    col("customerID").cast("string"),
    col("franchiseID").cast("string"),
    col("dateTime").cast("string"),
    col("product"),
    col("quantity").cast("bigInt"),
    col("unitPrice").cast("double"),
    (col("unitPrice") * col("quantity")).cast("bigInt").alias("totalPrice"),

)
df_casted.limit(5).display()

##Add/Drop de nueva columna

In [0]:
df_test = df_casted.withColumn("NuevaColuminta",lit("NuevaColumna_Literal!!!"))

df_new = df_test.drop("NuevaColuminta")

df_new.limit(5).display()